In [ ]:
from robust_gcn_structure.certification import certify
from robust_gcn_structure.utils import load_npz
from matplotlib import pyplot as plt
import torch
import numpy as np
from tqdm.autonotebook import tqdm
from multiprocessing import Pool

In [ ]:
dataset = "citeseer"
robust_gcn = True  # Whether to load weights for GCN trained with the approach by [Zügner and Günnemann 2019

local_budget = 3
global_budget = 5

In [ ]:
# In our experiments, the solver ECOS fails with an error in some instances, which does not happen with CPLEX.
# In case of an error we report non-robustness, therefore ECOS numbers are slightly lower than the CPLEX numbers
# reported in the paper.
solver = "ECOS"  # CPLEX is faster but is proprietary and not installed by default.
max_iters = 250
tolerance = 1e-2
kwargs = {
    'tolerance': tolerance,
    'max_iter': max_iters
}

In [ ]:
# Load data
A, X, z = load_npz(f'../datasets/{dataset}.npz')
A = A + A.T
A[A > 1] = 1
A.setdiag(0)

X = (X>0).astype("float32")
z = z.astype("int64")
N, D = X.shape

# Load weights
weight_path = f"../pretrained_weights/{dataset}"
if robust_gcn:
    weight_path = f"{weight_path}_robust_gcn.pkl"
else:
    weight_path = f"{weight_path}_gcn.pkl"

state_dict = torch.load(weight_path, map_location="cpu")

weights = [v for k,v in state_dict.items() if "weight" in k and "conv" in k]
biases = [v for k,v in state_dict.items() if "bias" in k and "conv" in k]

W1, W2 = [w.cpu().detach().numpy() for w in weights]
b1, b2 = [b.cpu().detach().numpy() for b in biases]

shapes = [x.shape[0] for x in biases]
num_hidden = len(shapes) - 1
if num_hidden > 1:
    raise NotImplementedError("Only one hidden layer is supported.")

weight_list = [W1, b1, W2, b2]



In [ ]:
# This gives exactly the 500 nodes that we sampled for our experiments.
np.random.seed(481516)
eval_nodes = np.random.choice(np.arange(0, A.shape[0]), 500, replace=False)

In [ ]:
def fct(target_node):
    return certify(target_node, A, X, weight_list, z,
                      local_changes=local_budget,
                      global_changes=global_budget,
                      solver=solver, eval_class=None,
                      use_predicted_class=True, **kwargs)
    return res

In [ ]:
processes = 10

In [ ]:
pool = Pool(processes=processes)

In [ ]:
robustness_results = []
errors = []
with tqdm(total=len(eval_nodes)) as pbar:
    for i, res in enumerate(pool.imap_unordered(fct, eval_nodes)):
        robustness_results.append(res['all_robust'])
        infs = False
        err = False
        for k,v in res.items():
            if type(v) != dict:
                continue
            if 'error' in v:
                err = err or v['error']
        infeasible.append(infs)
        errors.append(err)

        pbar.update()
        pbar.set_postfix(robust=np.mean(robustness_results), 
                         errors=np.mean(errors))

In [ ]:
np.mean(robustness_results) + np.mean(errors)

In [ ]:
np.mean(robustness_results)